In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]


In [3]:
data_train_path = "Dataset/data_train.csv"
data_test_path = "Dataset/data_target_users_test.csv"

In [4]:
df = pd.read_csv(data_train_path)
df_test = pd.read_csv(data_test_path)

df.columns = ["UserID", "ItemID", "Interaction"]
df_test.columns = ["UserID"]

In [5]:
df

,UserID,ItemID,Interaction
0,1,7,1.0
1,1,15,1.0
2,1,16,1.0
3,1,133,1.0
4,1,161,1.0
...,...,...,...
478725,13024,13605,1.0
478726,13024,13823,1.0
478727,13024,15122,1.0
478728,13024,18185,1.0


In [6]:
metric_to_optimize="MAP"
cutoff_to_optimize=10

In [7]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample


urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [10]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = []
for u in urm_validation:
    evaluator_validation.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2581 (20.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2598 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2621 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2601 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2649 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2614 (20.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2608 (20.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2586 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2593 

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2146 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2177 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2157 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2132 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2149 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2129 (16.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2228 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2183 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2140 

In [12]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

recommenderClass = ItemKNNCFRecommender



In [13]:
import optuna

# Replace the following with your MySQL database connection details
host = "db-mysql-nyc3-14396-do-user-15539286-0.c.db.ondigitalocean.com"
port = 25060
database_name = "recsys_optuna"
username = "doadmin"
password = "AVNS_EIQ1D-GbDGJWZPsN3Mt"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
storage = optuna.storages.RDBStorage(url=mysql_url)

In [14]:
rp3_study = optuna.load_study(study_name="RP3_Hybrid_last", storage=storage)
p3_study = optuna.load_study(study_name="P3_Hybrid_last", storage=storage)
slim_study = optuna.load_study(study_name="SLIM_Hybrid_last", storage=storage)
knn_study = optuna.load_study(study_name="KNN_Hybrid_last", storage=storage)
mf_study = optuna.load_study(study_name="NMF_Hybrid_last", storage=storage)


In [15]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender
from Recommenders.NonPersonalizedRecommender.TopPop import TopPop

slims = []
rp3s = []
p3s = []
knns = []
nmfs = []
tops = []

for i in range(k_fold):
    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train[i])
    slim.fit(workers=8, **slim_study.best_params)
    slims.append(slim)

    rp3 = RP3betaRecommender(urm_train[i])
    rp3.fit(**rp3_study.best_params)
    rp3s.append(rp3)

    p3 = P3alphaRecommender(urm_train[i])
    p3.fit(**p3_study.best_params)
    p3s.append(p3)

    knn = ItemKNNCFRecommender(urm_train[i])
    knn.fit(**knn_study.best_params)
    knns.append(knn)

    top = TopPop(urm_train[i])
    top.fit()
    tops.append(top)

    #nmf = NMFRecommender(urm_train[i])
    #nmf.fit(**mf_study.best_params)
    #nmfs.append(nmf)

SLIMElasticNetRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 332 ( 1.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:29<00:00, 744.76it/s] 


RP3betaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
RP3betaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9472.88 column/sec. Elapsed time 2.35 sec
P3alphaRecommender: URM Detected 508 ( 4.0%) users with no interactions.
P3alphaRecommender: URM Detected 332 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10479.71 column/sec. Elapsed time 2.12 sec
ItemKNNCFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13554.67 column/sec. Elapsed time 1.64 sec
NMFRecommender: URM Detected 508 ( 4.0%) users with no interactions.
NMFRecommender: URM Detected 332 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 14.13 sec
SLIMElasticNetRecommender: URM Detecte

100%|█████████▉| 22216/22222 [00:30<00:00, 736.89it/s] 


RP3betaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
RP3betaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9596.34 column/sec. Elapsed time 2.32 sec
P3alphaRecommender: URM Detected 440 ( 3.5%) users with no interactions.
P3alphaRecommender: URM Detected 325 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10274.26 column/sec. Elapsed time 2.16 sec
ItemKNNCFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13503.48 column/sec. Elapsed time 1.65 sec
NMFRecommender: URM Detected 440 ( 3.5%) users with no interactions.
NMFRecommender: URM Detected 325 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 10.50 sec
SLIMElasticNetRecommender: URM Detecte

...

RP3betaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
RP3betaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9412.71 column/sec. Elapsed time 2.36 sec
P3alphaRecommender: URM Detected 471 ( 3.7%) users with no interactions.
P3alphaRecommender: URM Detected 362 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10484.33 column/sec. Elapsed time 2.12 sec
ItemKNNCFRecommender: URM Detected 471 ( 3.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 362 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 13941.16 column/sec. Elapsed time 1.59 sec
NMFRecommender: URM Detected 471 ( 3.7%) users with no interactions.
NMFRecommender: URM Detected 362 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 9.43 sec
SLIMElasticNetRecommender: URM Detected

100%|█████████▉| 22216/22222 [00:30<00:00, 725.10it/s]


RP3betaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
RP3betaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9456.40 column/sec. Elapsed time 2.35 sec
P3alphaRecommender: URM Detected 478 ( 3.8%) users with no interactions.
P3alphaRecommender: URM Detected 329 ( 1.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10419.18 column/sec. Elapsed time 2.13 sec
ItemKNNCFRecommender: URM Detected 478 ( 3.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
Similarity column 22222 (100.0%), 13797.02 column/sec. Elapsed time 1.61 sec
NMFRecommender: URM Detected 478 ( 3.8%) users with no interactions.
NMFRecommender: URM Detected 329 ( 1.5%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 13.71 sec
SLIMElasticNetRecommender: URM Detecte

100%|█████████▉| 22216/22222 [00:30<00:00, 724.01it/s]


RP3betaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
RP3betaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 9561.45 column/sec. Elapsed time 2.32 sec
P3alphaRecommender: URM Detected 460 ( 3.6%) users with no interactions.
P3alphaRecommender: URM Detected 348 ( 1.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 10442.94 column/sec. Elapsed time 2.13 sec
ItemKNNCFRecommender: URM Detected 460 ( 3.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
Similarity column 22222 (100.0%), 13834.26 column/sec. Elapsed time 1.61 sec
NMFRecommender: URM Detected 460 ( 3.6%) users with no interactions.
NMFRecommender: URM Detected 348 ( 1.6%) items with no interactions.
NMFRecommender: Computing NMF decomposition...
NMFRecommender: Computing NMF decomposition... done in 11.09 sec


In [25]:
from Recommenders.NonPersonalizedRecommender import TopPop

tops = []
for i in range(k_fold):
    top = TopPop(urm_train[i])
    top.fit()
    tops.append(top)

TopPopRecommender: URM Detected 508 ( 4.0%) users with no interactions.
TopPopRecommender: URM Detected 332 ( 1.5%) items with no interactions.
TopPopRecommender: URM Detected 440 ( 3.5%) users with no interactions.
TopPopRecommender: URM Detected 325 ( 1.5%) items with no interactions.
TopPopRecommender: URM Detected 479 ( 3.8%) users with no interactions.
TopPopRecommender: URM Detected 345 ( 1.6%) items with no interactions.
TopPopRecommender: URM Detected 451 ( 3.6%) users with no interactions.
TopPopRecommender: URM Detected 311 ( 1.4%) items with no interactions.
TopPopRecommender: URM Detected 449 ( 3.6%) users with no interactions.
TopPopRecommender: URM Detected 320 ( 1.4%) items with no interactions.
TopPopRecommender: URM Detected 460 ( 3.6%) users with no interactions.
TopPopRecommender: URM Detected 328 ( 1.5%) items with no interactions.
TopPopRecommender: URM Detected 426 ( 3.4%) users with no interactions.
TopPopRecommender: URM Detected 344 ( 1.5%) items with no intera

In [31]:
hybrids = []

for i in range(k_fold):
    hybrid = GeneralizedLinearHybridRecommender(urm_train[i], recommenders=[slims[i], rp3s[i], p3s[i], knns[i], tops[i]])
    hybrids.append(hybrid)

SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 508 ( 4.0%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 332 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 440 ( 3.5%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 325 ( 1.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 479 ( 3.8%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 345 ( 1.6%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 451 ( 3.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 311 ( 1.4%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 449 (

In [27]:
study = optuna.create_study(study_name="LINEAR_TOPPOP_Hybrid_last", direction="maximize", storage=storage, load_if_exists=True, sampler=optuna.samplers.TPESampler())

[I 2024-01-10 12:44:04,031] A new study created in RDB with name: LINEAR_TOPPOP_Hybrid_last


In [28]:
def objective(trial):

    alphas = []
    w_slim = trial.suggest_float("w_slim", 0.8, 1)
    w_knn = trial.suggest_float("w_knn", 0, 0.1)
    w_p3 = trial.suggest_float("w_p3", 0, 0.05)
    w_rp3 = trial.suggest_float("w_rp3", 0.4, 0.7)
    w_top = trial.suggest_float("w_top", 0, 0.0001)
        
    alphas = [w_slim, w_rp3, w_p3, w_knn, w_top]
    MAP = 0
    for i in range(k_fold):
        recommender = hybrids[i]
        recommender.fit(alphas=alphas)

        result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
        print(f"MAP: {result_dict['MAP'].item()}, RECALL: {result_dict['RECALL'].item()}")
        MAP += result_dict["MAP"].item()
    
    MAP /= k_fold
    return MAP

In [29]:
study.optimize(objective, n_trials=200)

EvaluatorHoldout: Processed 10057 (100.0%) in 6.61 sec. Users per second: 1522
MAP: 0.030756576403389836, RECALL: 0.12262237320871533
EvaluatorHoldout: Processed 10040 (100.0%) in 6.75 sec. Users per second: 1487
MAP: 0.03083824622146368, RECALL: 0.12212385342902911
EvaluatorHoldout: Processed 10017 (100.0%) in 5.98 sec. Users per second: 1674
MAP: 0.030357697469856986, RECALL: 0.11835629816967372
EvaluatorHoldout: Processed 10037 (100.0%) in 6.37 sec. Users per second: 1577
MAP: 0.03059204751941662, RECALL: 0.12128811029293887
EvaluatorHoldout: Processed 9989 (100.0%) in 6.34 sec. Users per second: 1576
MAP: 0.030984638658079662, RECALL: 0.12485952139931035
EvaluatorHoldout: Processed 10024 (100.0%) in 6.04 sec. Users per second: 1659
MAP: 0.031257014910247335, RECALL: 0.12134101518908484
EvaluatorHoldout: Processed 10030 (100.0%) in 6.80 sec. Users per second: 1475
MAP: 0.031061335833768848, RECALL: 0.1259983042202435
EvaluatorHoldout: Processed 10052 (100.0%) in 7.48 sec. Users per 

[I 2024-01-10 12:48:13,211] Trial 0 finished with value: 0.030854162657913464 and parameters: {'w_slim': 0.897329749016747, 'w_knn': 0.010286981589046685, 'w_p3': 0.030781954806672247, 'w_rp3': 0.6690525599190291, 'w_top': 5.7189438830901885e-05}. Best is trial 0 with value: 0.030854162657913464.


EvaluatorHoldout: Processed 10057 (100.0%) in 6.00 sec. Users per second: 1678
MAP: 0.030943250456525045, RECALL: 0.12057194788196438
EvaluatorHoldout: Processed 10040 (100.0%) in 6.63 sec. Users per second: 1515
MAP: 0.031033504869411553, RECALL: 0.11929948042867106
EvaluatorHoldout: Processed 10017 (100.0%) in 7.26 sec. Users per second: 1379
MAP: 0.030501306509093436, RECALL: 0.11744841027876991
EvaluatorHoldout: Processed 10037 (100.0%) in 7.06 sec. Users per second: 1422
MAP: 0.030898872584137448, RECALL: 0.1208228513710756
EvaluatorHoldout: Processed 9989 (100.0%) in 7.55 sec. Users per second: 1322
MAP: 0.03125277885038649, RECALL: 0.1251565392416361
EvaluatorHoldout: Processed 10024 (100.0%) in 6.73 sec. Users per second: 1489
MAP: 0.031411620048392055, RECALL: 0.12069647024003073
EvaluatorHoldout: Processed 10030 (100.0%) in 6.59 sec. Users per second: 1521
MAP: 0.03130466141258791, RECALL: 0.12430657883153393
EvaluatorHoldout: Processed 10052 (100.0%) in 7.00 sec. Users per s

[I 2024-01-10 12:49:34,856] Trial 1 finished with value: 0.03105842184585539 and parameters: {'w_slim': 0.821382308541623, 'w_knn': 0.060012770765681116, 'w_p3': 0.02219868024113902, 'w_rp3': 0.5994081151901731, 'w_top': 2.900410893780414e-06}. Best is trial 1 with value: 0.03105842184585539.


EvaluatorHoldout: Processed 10057 (100.0%) in 7.00 sec. Users per second: 1436
MAP: 0.030902474940458627, RECALL: 0.12126431740162638
EvaluatorHoldout: Processed 10040 (100.0%) in 7.22 sec. Users per second: 1390
MAP: 0.030894821507620535, RECALL: 0.12105138888834273
EvaluatorHoldout: Processed 10017 (100.0%) in 7.22 sec. Users per second: 1387
MAP: 0.030631454305458657, RECALL: 0.11924762496955113
EvaluatorHoldout: Processed 10037 (100.0%) in 6.96 sec. Users per second: 1443
MAP: 0.030823295078052818, RECALL: 0.12191147747892936
EvaluatorHoldout: Processed 9989 (100.0%) in 7.07 sec. Users per second: 1412
MAP: 0.031103904771439338, RECALL: 0.12567834279083287
EvaluatorHoldout: Processed 10024 (100.0%) in 7.09 sec. Users per second: 1415
MAP: 0.03135855692370086, RECALL: 0.1225184342488638
EvaluatorHoldout: Processed 10030 (100.0%) in 7.39 sec. Users per second: 1357
MAP: 0.031170476665242607, RECALL: 0.1263881612547348
EvaluatorHoldout: Processed 10052 (100.0%) in 7.10 sec. Users per 

...

EvaluatorHoldout: Processed 10057 (100.0%) in 5.25 sec. Users per second: 1915
MAP: 0.03101049415159004, RECALL: 0.12109156214681194
EvaluatorHoldout: Processed 10040 (100.0%) in 6.10 sec. Users per second: 1645
MAP: 0.031116786820970348, RECALL: 0.1200712787836566
EvaluatorHoldout: Processed 10017 (100.0%) in 6.28 sec. Users per second: 1595
MAP: 0.030673878216555886, RECALL: 0.11766543714064355
EvaluatorHoldout: Processed 10037 (100.0%) in 6.52 sec. Users per second: 1539
MAP: 0.031069151283109917, RECALL: 0.12120268711966442
EvaluatorHoldout: Processed 9989 (100.0%) in 7.07 sec. Users per second: 1414
MAP: 0.03120604887598604, RECALL: 0.1251044010406933
EvaluatorHoldout: Processed 10024 (100.0%) in 5.54 sec. Users per second: 1811
MAP: 0.031436991696119986, RECALL: 0.12140656198688203
EvaluatorHoldout: Processed 10030 (100.0%) in 5.50 sec. Users per second: 1822
MAP: 0.03143183544604312, RECALL: 0.1250782693637974
EvaluatorHoldout: Processed 10052 (100.0%) in 6.25 sec. Users per sec

[I 2024-01-10 16:20:49,622] Trial 160 finished with value: 0.031141273521295233 and parameters: {'w_slim': 0.8043427087967256, 'w_knn': 0.09959162158745923, 'w_p3': 0.0360223030768756, 'w_rp3': 0.6932587749314254, 'w_top': 8.415604313977257e-06}. Best is trial 146 with value: 0.031162907974085812.


EvaluatorHoldout: Processed 10057 (100.0%) in 5.97 sec. Users per second: 1683
MAP: 0.031056612230918802, RECALL: 0.12162206345725918
EvaluatorHoldout: Processed 10040 (100.0%) in 5.36 sec. Users per second: 1873
MAP: 0.031136260513501785, RECALL: 0.11989701523042968
EvaluatorHoldout: Processed 10017 (100.0%) in 6.13 sec. Users per second: 1633
MAP: 0.030629905351379043, RECALL: 0.11820637559001267
EvaluatorHoldout: Processed 10037 (100.0%) in 6.51 sec. Users per second: 1541
MAP: 0.031086997948859333, RECALL: 0.12139149999796686
EvaluatorHoldout: Processed 9989 (100.0%) in 6.75 sec. Users per second: 1480
MAP: 0.031223321844505377, RECALL: 0.12520847036431806
EvaluatorHoldout: Processed 10024 (100.0%) in 6.22 sec. Users per second: 1612
MAP: 0.03147962746551157, RECALL: 0.12163606861820851


[W 2024-01-10 16:21:40,627] Trial 161 failed with parameters: {'w_slim': 0.8000573790899062, 'w_knn': 0.09764249069604387, 'w_p3': 0.034771929091704984, 'w_rp3': 0.6997425122141736, 'w_top': 1.45505426395584e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/japo/miniconda3/envs/RecSysFramework/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_387424/3544739749.py", line 16, in objective
    result_dict, _ = evaluator_validation[i].evaluateRecommender(recommender)
  File "/home/japo/RecSys-Challenge/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/home/japo/RecSys-Challenge/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = recommender_object.recommend(test

KeyboardInterrupt: 

In [32]:
best_MAP_test = 0

for i in range(k_fold):
    recommender = hybrids[i]

    alphas = [study.best_params['w_slim'], study.best_params['w_rp3'], study.best_params['w_p3'], study.best_params['w_knn'], study.best_params['w_top']]

    recommender.fit(alphas)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test

EvaluatorHoldout: Processed 10492 (100.0%) in 5.57 sec. Users per second: 1883
EvaluatorHoldout: Processed 10461 (100.0%) in 5.30 sec. Users per second: 1973
EvaluatorHoldout: Processed 10481 (100.0%) in 6.01 sec. Users per second: 1744
EvaluatorHoldout: Processed 10506 (100.0%) in 5.91 sec. Users per second: 1778
EvaluatorHoldout: Processed 10489 (100.0%) in 6.99 sec. Users per second: 1501
EvaluatorHoldout: Processed 10509 (100.0%) in 5.44 sec. Users per second: 1933
EvaluatorHoldout: Processed 10410 (100.0%) in 6.08 sec. Users per second: 1713
EvaluatorHoldout: Processed 10455 (100.0%) in 6.24 sec. Users per second: 1675
EvaluatorHoldout: Processed 10498 (100.0%) in 5.54 sec. Users per second: 1896
EvaluatorHoldout: Processed 10451 (100.0%) in 5.82 sec. Users per second: 1796


0.03939730496463813

In [33]:
slims = []
rp3s = []
p3s = []
knns = []
nmfs = []
hybrids = []
tops = []

for i in range(k_fold):
    slim = MultiThreadSLIM_SLIMElasticNetRecommender(urm_train_validation[i])
    slim.fit(workers=8, **slim_study.best_params)
    slims.append(slim)

    rp3 = RP3betaRecommender(urm_train_validation[i])
    rp3.fit(**rp3_study.best_params)
    rp3s.append(rp3)

    p3 = P3alphaRecommender(urm_train_validation[i])
    p3.fit(**p3_study.best_params)
    p3s.append(p3)

    knn = ItemKNNCFRecommender(urm_train_validation[i])
    knn.fit(**knn_study.best_params)
    knns.append(knn)

    top = TopPop(urm_train_validation[i])
    top.fit()
    tops.append(top)

    hybrid = GeneralizedLinearHybridRecommender(urm_train_validation[i], recommenders=[slims[i], rp3s[i], p3s[i], knns[i], tops[i]])
    alphas = [study.best_params['w_slim'], study.best_params['w_rp3'], study.best_params['w_p3'], study.best_params['w_knn'], study.best_params['w_top']]
    hybrid.fit(alphas=alphas)
    hybrids.append(hybrid)

    #nmf = NMFRecommender(urm_train[i])
    #nmf.fit(**mf_study.best_params)
    #nmfs.append(nmf)




SLIMElasticNetRecommender: URM Detected 249 ( 2.0%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 113 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:36<00:00, 616.50it/s]


RP3betaRecommender: URM Detected 249 ( 2.0%) users with no interactions.
RP3betaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7933.90 column/sec. Elapsed time 2.80 sec
P3alphaRecommender: URM Detected 249 ( 2.0%) users with no interactions.
P3alphaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8462.64 column/sec. Elapsed time 2.63 sec
ItemKNNCFRecommender: URM Detected 249 ( 2.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 113 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12132.72 column/sec. Elapsed time 1.83 sec
TopPopRecommender: URM Detected 249 ( 2.0%) users with no interactions.
TopPopRecommender: URM Detected 113 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 249 ( 2.0%) users with no interactions.
SLIMElasticNetRP3betaP3alphaIt

100%|█████████▉| 22216/22222 [00:35<00:00, 630.95it/s]


RP3betaRecommender: URM Detected 191 ( 1.5%) users with no interactions.
RP3betaRecommender: URM Detected 101 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8103.98 column/sec. Elapsed time 2.74 sec
P3alphaRecommender: URM Detected 191 ( 1.5%) users with no interactions.
P3alphaRecommender: URM Detected 101 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8778.41 column/sec. Elapsed time 2.53 sec
ItemKNNCFRecommender: URM Detected 191 ( 1.5%) users with no interactions.
ItemKNNCFRecommender: URM Detected 101 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12082.47 column/sec. Elapsed time 1.84 sec
TopPopRecommender: URM Detected 191 ( 1.5%) users with no interactions.
TopPopRecommender: URM Detected 101 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 191 ( 1.5%) users with no interactions.
SLIMElasticNetRP3betaP3alphaIt

...

RP3betaRecommender: URM Detected 218 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 141 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 8308.12 column/sec. Elapsed time 2.67 sec
P3alphaRecommender: URM Detected 218 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 141 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 9162.09 column/sec. Elapsed time 2.43 sec
ItemKNNCFRecommender: URM Detected 218 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 141 ( 0.6%) items with no interactions.
Similarity column 22222 (100.0%), 12785.73 column/sec. Elapsed time 1.74 sec
TopPopRecommender: URM Detected 218 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 141 ( 0.6%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 218 ( 1.7%) users with no interactions.
SLIMElasticNetRP3betaP3alphaIt

100%|█████████▉| 22216/22222 [00:35<00:00, 621.03it/s]


RP3betaRecommender: URM Detected 214 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 117 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7743.80 column/sec. Elapsed time 2.87 sec
P3alphaRecommender: URM Detected 214 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 117 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 8761.45 column/sec. Elapsed time 2.54 sec
ItemKNNCFRecommender: URM Detected 214 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 117 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 12474.80 column/sec. Elapsed time 1.78 sec
TopPopRecommender: URM Detected 214 ( 1.7%) users with no interactions.
TopPopRecommender: URM Detected 117 ( 0.5%) items with no interactions.
SLIMElasticNetRP3betaP3alphaItemKNNCFTopPopHybridRecommender: URM Detected 214 ( 1.7%) users with no interactions.
SLIMElasticNetRP3betaP3alphaIt

100%|█████████▉| 22216/22222 [00:36<00:00, 609.62it/s]


RP3betaRecommender: URM Detected 217 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 113 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7699.51 column/sec. Elapsed time 2.89 sec
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 113 ( 0.5%) items with no interactions.


In [ ]:
best_MAP_test = 0
print(study.best_value)
for i in range(k_fold):
    recommender =hybrids[i]
    recommender.fit(**study.best_params)

    result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

    best_MAP_test += result_dict["MAP"].item()

best_MAP_test /= k_fold
best_MAP_test